# A training log

- `step 1000`: train loss 5.9037, `val loss 5.8993`

![At 1000 epochs](./training-log/1000-epochs.png "1000 Epochs")

- `step 1500`: train loss 5.6095, `val loss 5.5964`
- interestingly, more gibberish. will run more epochs before checking again.

![At 1500 epochs](./training-log/1500-epochs.png "1500 Epochs")

- `step 3000`: train loss 5.0643, `val loss 5.0579`
- seems very "news"-y?

![At 3000 epochs](./training-log/3000-epochs.png "3000 Epochs")

- `step 5500`: train loss 4.6092, `val loss 4.6042`
- output is looking more coherent, but still very "news"-y
- let's wait until 10k epochs and then see if we need to bump up the learning rate

![At 5500 epochs](./training-log/5500-epochs.png "5500 Epochs")